# Framework

## Initialization

In [ ]:
import pandas as pd
import os
import time
import subprocess

FILE_DIR = '../media/files'
OUT_DIR = os.path.join(FILE_DIR, 'processed')

In [ ]:
import sc3nb as scn

In [ ]:
# start scsynth
# sc = scn.startup(start_sclang=False)
sc = scn.startup()

# connect scsynth to the system playback
!jack_connect "SuperCollider:out_1" "system:playback_1"
!jack_connect "SuperCollider:out_2" "system:playback_2"

# multiclient setup by default
sc.server.dump_tree()

In [ ]:
sc.exit()

## Sonification definition

In [ ]:
import panson as ps
from panson import bundle

### Message-style (explicit ID allocation)

#### Explicit bundling

In [ ]:
from sc3nb.osc.osc_communication import Bundler

# message style (explicit ID allocation)
class AU04ContinuousSonification(ps.Sonification):
    
    def _initialize(self):
        bundler = Bundler()
        bundler.add(
            0,
            "/d_load",
            ['/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef']
        )
        return bundler
    
    def _start(self):
        bundler = Bundler()
        self.au4_node_id = self._s.node_ids.allocate(1)[0]        
        bundler.add(0, "/s_new", ["s2", self.au4_node_id, 0, 0, "amp", 0])
        return bundler
    
    def _stop(self, server):
        bundler = Bundler()
        self._s.node_ids.free([self.au4_node_id])    # actually this does nothing
        bundler.add(0, "/g_freeAll", [0])
        return bundler
    
    def _process(self, row):
        bundler = Bundler()
        # only "max" should be enough (to clip the top part to 0.3)
        amp = scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax")
        # map the intensity of the AU in one octave range
        freq = scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
        bundler.add(0, "/n_set", [self.au4_node_id, "amp", amp, "freq", freq])
        return bundler

In [ ]:
from sc3nb.osc.osc_communication import Bundler

# # message style (explicit ID allocation)
class AU04ContinuousSonification(ps.Sonification):
    
    def _initialize(self):
        with Bundler(send_on_exit=False) as bundler:
            self._s.msg(
                "/d_load",
                ['/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef'],
                bundle=True
            )
        return bundler
    
    def _start(self):
        with Bundler(send_on_exit=False) as bundler:
            self.au4_node_id = self._s.node_ids.allocate(1)[0]
            self._s.msg("/s_new", ["s2", self.au4_node_id, 0, 0, "amp", 0], bundle=True)
        return bundler
    
    def _stop(self):
        with Bundler(send_on_exit=False) as bundler:
            self._s.node_ids.free([self.au4_node_id])    # actually this does nothing
            self._s.msg("/g_freeAll", [0], bundle=True)
        return bundler
    
    def _process(self, row):
        with Bundler(send_on_exit=False) as bundler:
            # only "max" should be enough (to clip the top part to 0.3)
            amp = scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax")
            # map the intensity of the AU in one octave range
            freq = scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
            self._s.msg("/n_set", [self.au4_node_id, "amp", amp, "freq", freq], bundle=True)
        return bundler

#### Implicit bundling

TODO: allow the user to remove flag bundle=True???

In [ ]:
from sc3nb.osc.osc_communication import Bundler

class AU04ContinuousSonification(ps.Sonification):
    
    @bundle
    def _initialize(self):
        self._s.msg(
            "/d_load",
            ['/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef'],
            bundle=True
        )
    
    @bundle
    def _start(self):
        self.au4_node_id = self._s.node_ids.allocate(1)[0]
        self._s.msg("/s_new", ["s2", self.au4_node_id, 0, 0, "amp", 0], bundle=True)
    
    @bundle
    def _stop(self):
        self._s.node_ids.free([self.au4_node_id])    # actually this does nothing
        self._s.msg("/g_freeAll", [0], bundle=True)
    
    @bundle
    def _process(self, row):
        # only "max" should be enough (to clip the top part to 0.3)
        amp = scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax")
        # map the intensity of the AU in one octave range
        freq = scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
        self._s.msg("/n_set", [self.au4_node_id, "amp", amp, "freq", freq], bundle=True)

### High-level style (implicit ID allocation)

#### Explicit bundling

In [ ]:
from sc3nb.osc.osc_communication import Bundler
from sc3nb import SynthDef


class AU04ContinuousSonification(ps.Sonification):

    def _initialize(self):
        with Bundler(send_on_exit=False) as bundler:
            # self._s.load_synthdefs()
            SynthDef.load("/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef")
        return bundler

    def _start(self):
        with Bundler(send_on_exit=False) as bundler:
            self.synth = scn.Synth("s2", {"amp": 0})
        return bundler
    
    def _stop(self):
        with Bundler(send_on_exit=False) as bundler:
            self._s.free_all()
        return bundler
    
    def _process(self, row):
        with Bundler(send_on_exit=False) as bundler:
            self.synth.set(
                # only "max" should be enough (to clip the top part to 0.3)
                "amp", scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax"),
                # map the intensity of the AU in one octave range
                "freq", scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
            )
        return bundler

#### Implicit bundling

In [ ]:
from sc3nb import SynthDef

# a (implicit ID allocation)
class AU04ContinuousSonification(ps.Sonification):
    
    def __init__(self):
        super().__init__()
        self.amp = 1
    
    @bundle
    def _initialize(self):
        SynthDef.load("/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef")

    @bundle
    def _start(self):
        self.synth = scn.Synth("s2", {"amp": 0})
    
    @bundle
    def _stop(self):
        self._s.free_all()
    
    @bundle
    def _process(self, row):
        self.synth.set(
            # only "max" should be enough (to clip the top part to 0.3)
            "amp", self.amp * scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax"),
            # map the intensity of the AU in one octave range
            "freq", scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
        )

In [ ]:
son = AU04ContinuousSonification()

In [ ]:
son.set('amp', 0.5)

In [ ]:
son.get('amp')

### Realtime usage

In [ ]:
sc.server.latency = 0.1

#### Offline

In [ ]:
df = pd.read_csv(os.path.join(OUT_DIR, "phone.csv"), sep=r',\s*', engine='python')
df.head()

In [ ]:
dp = ps.DataPlayer(son).load(df)
dp   # TODO: display

In [ ]:
dp.rate = 1

In [ ]:
dp.play()
# dp._play()

In [ ]:
dp.pause()

In [ ]:
dp.seek(0.3)

#### Online

In [ ]:
# import logging
# logging.basicConfig(level=logging.DEBUG)

In [ ]:
import csv

FIFO = os.path.join(FILE_DIR, 'pipe.csv')

def data_generator():
    with open(FIFO, 'r') as fifo:
        # the reader attempts to execute fifo.readline()
        # which blocks if there are no lines
        reader = csv.reader(fifo, skipinitialspace=True)
        
        header = next(reader)
        
        # the loop ends when the pipe is closed from the writing side
        for i, row in enumerate(reader):
            series = pd.Series(row, header, dtype='float', name=i)
            yield series

In [ ]:
CONTAINER_NAME = 'openface'

# base directory of the container
CONTAINER_BASE_DIR = '/home/openface-build'
# directory with executalbles in the container
CONTAINER_BIN_DIR = os.path.join(CONTAINER_BASE_DIR, 'build/bin')

FILE_DIR = '../media/files'
OUT_DIR = os.path.join(FILE_DIR, 'processed')

CONTAINER_FILE_DIR = os.path.join(CONTAINER_BASE_DIR, 'files')
CONTAINER_OUT_DIR = os.path.join(CONTAINER_FILE_DIR, 'processed')

CONTAINER_EXECUTABLE = os.path.join(CONTAINER_BIN_DIR, 'FeatureExtraction')

def feature_extraction_online(pipe='files/pipe'):
    command = [
        'docker', 'exec', CONTAINER_NAME, CONTAINER_EXECUTABLE,
        '-device', # use default device
        '-pose', '-gaze', '-aus',
        # '-tracked'
        '-of', pipe
    ]
    
    # capture and combine stdout and stderr into one stream and set as text stream
    proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)    

    print('Starting real-time analysis...')
    print('Open the pipe from the read side to start the feature stream')
    
    return proc

def kill_feature_extraction_online():
    # !docker exec -it openface pkill FeatureExt
    command = ['docker', 'exec', CONTAINER_NAME, 'pkill', 'FeatureExt']
    subprocess.run(command)

In [ ]:
kill_feature_extraction_online()

In [ ]:
rtdp = ps.RTDataPlayer(data_generator, son)
rtdp

In [ ]:
rtdp.add_listen_hook(feature_extraction_online)
rtdp.add_close_hook(kill_feature_extraction_online)

In [ ]:
rtdp.listen()

In [ ]:
rtdp.close()

In [ ]:
rtdp.record_start()

In [ ]:
rtdp.record_stop()

In [ ]:
rtdp.log_start()

In [ ]:
df = rtdp.log_stop()

### NRT usage

In [ ]:
dp.export("score.wav")